# Spark SQL Examples

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
path = "/FileStore/tables/riders.csv"
df_riders = spark.read \
    .option("inferSchema", "true") \
    .option("header", "false") \
    .option("sep", ",") \
    .format("csv") \
    .load(path) \
    .withColumnRenamed("_c0","rider_id") \
    .withColumnRenamed("_c1","firstName")\
    .withColumnRenamed("_c2","lastName")\
    .withColumnRenamed("_c3","address")\
    .withColumnRenamed("_c4","birthday")\
    .withColumnRenamed("_c5","startDate")\
    .withColumnRenamed("_c6","endDate")\
    .withColumnRenamed("_c7","isMember")

# Create a View And Run Queries

The code below creates a temporary view against which you can run SQL queries.

In [ ]:
df_riders.createOrReplaceTempView("riders")

In [ ]:
%sql
SELECT * FROM riders LIMIT 2

rider_id,firstName,lastName,address,birthday,startDate,endDate,isMember
1000,Diana,Clark,1200 Alyssa Squares,1989-02-13T00:00:00.000+0000,2019-04-23T00:00:00.000+0000,null,true
1001,Jennifer,Smith,397 Diana Ferry,1976-08-10T00:00:00.000+0000,2019-11-01T00:00:00.000+0000,2020-09-01T00:00:00.000+0000,true


In [ ]:
%sql 

SELECT COUNT(*) FROM riders 

count(1)
75000


In [ ]:
%sql
SELECT rider_id, firstName, lastName, address, birthday, startDate, endDate, isMember
FROM riders 
WHERE rider_id == '1046'

rider_id,firstName,lastName,address,birthday,startDate,endDate,isMember
1046,Brianna,Flores,351 Bill Curve,2005-11-09T00:00:00.000+0000,2020-07-08T00:00:00.000+0000,null,true


In [ ]:
%sql

SELECT count(*), isMember
FROM riders 
GROUP BY isMember
         

count(1),isMember
60124,true
14876,false


# User Defined Functions

In [ ]:
spark.udf.register("get_day", lambda x: x.day if not x is None else 0, IntegerType())

Out[6]: <function __main__.<lambda>(x)>

In [ ]:
%sql 

SELECT *, get_day(startDate) AS day
FROM riders 
LIMIT 1

rider_id,firstName,lastName,address,birthday,startDate,endDate,isMember,day
1000,Diana,Clark,1200 Alyssa Squares,1989-02-13T00:00:00.000+0000,2019-04-23T00:00:00.000+0000,null,true,23


In [ ]:
count_members_per_day = spark.sql('''
    SELECT get_day(startDate) AS day, count(*) AS countMembers
    FROM riders
    WHERE isMember = true
    GROUP BY day
    ORDER BY day ASC
''')

In [ ]:
count_members_per_day.show()

+---+------------+
day|countMembers|
+---+------------+
 1| 1972|
 2| 1987|
 3| 1957|
 4| 2037|
 5| 2011|
 6| 2055|
 7| 1929|
 8| 1950|
 9| 1972|
 10| 2016|
 11| 2043|
 12| 2024|
 13| 1939|
 14| 1978|
 15| 1958|
 16| 1924|
 17| 1976|
 18| 1917|
 19| 1969|
 20| 1899|
+---+------------+
only showing top 20 rows

# Converting Results to Pandas

In [ ]:
count_members_per_day_pd = count_members_per_day.toPandas()

In [ ]:
display(count_members_per_day_pd)

day,countMembers
1,1972
2,1987
3,1957
4,2037
5,2011
6,2055
7,1929
8,1950
9,1972
10,2016
